<a href="https://colab.research.google.com/github/somabasavaiah/Data-Engineering-Projects/blob/main/HEALTHCARE_DATA_INTEGRATION_ETL_PROJECT_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [3]:
patients_data = {
    "patient_id": [101, 102, 103, 104],
    "name": ["John Doe", "Alice Smith", "Robert Lee", "Maria Gomez"],
    "age": [35, 42, 29, None],      # Missing value
    "gender": ["M", "F", "M", "F"]
}

df_patients = pd.DataFrame(patients_data)
clinical_data = {
    "record_id": [1, 2, 3, 4],
    "patient_id": [101, 102, 103, 105],  # 105 doesn't exist → quality issue
    "diagnosis": ["Diabetes", "Hypertension", "", "Asthma"],  # empty value
    "visit_date": ["2024-01-10", "2024-01-12", "2024-01-15", "2024-01-20"]
}

df_clinical = pd.DataFrame(clinical_data)

print("RAW PATIENT DATA:")
display(df_patients)

print("RAW CLINICAL DATA:")
display(df_clinical)


RAW PATIENT DATA:


,patient_id,name,age,gender
0,101,John Doe,35.0,M
1,102,Alice Smith,42.0,F
2,103,Robert Lee,29.0,M
3,104,Maria Gomez,NaN,F


RAW CLINICAL DATA:


,record_id,patient_id,diagnosis,visit_date
0,1,101,Diabetes,2024-01-10
1,2,102,Hypertension,2024-01-12
2,3,103,,2024-01-15
3,4,105,Asthma,2024-01-20


In [4]:
print("\n🔍 Running Data Quality Checks...")


missing_patients = df_patients.isnull().sum()
missing_clinical = df_clinical.isnull().sum()

print("\nMissing Values:")
print(missing_patients)
print(missing_clinical)


🔍 Running Data Quality Checks...

Missing Values:
patient_id    0
name          0
age           1
gender        0
dtype: int64
record_id     0
patient_id    0
diagnosis     0
visit_date    0
dtype: int64


In [6]:
invalid_ids = df_clinical[~df_clinical["patient_id"].isin(df_patients["patient_id"])]

print("\nInvalid Patient IDs:")
display(invalid_ids)
print("\n🧹 Cleaning Data...")


Invalid Patient IDs:


,record_id,patient_id,diagnosis,visit_date
3,4,105,Asthma,2024-01-20



🧹 Cleaning Data...


In [8]:
df_patients["age"].fillna(df_patients["age"].mean(), inplace=True)


df_clinical["diagnosis"].replace("", "Unknown", inplace=True)

df_clinical = df_clinical[df_clinical["patient_id"].isin(df_patients["patient_id"])]

print("CLEANED PATIENT DATA:")
display(df_patients)

print("CLEANED CLINICAL DATA:")
display(df_clinical)




print("\n🔗 Joining datasets...")

df_final = pd.merge(df_patients, df_clinical, on="patient_id", how="inner")

print("FINAL CURATED HEALTHCARE DATA:")
display(df_final)




output_path = "/content/curated_healthcare_data.csv"
df_final.to_csv(output_path, index=False)

print("\n📁 Data saved successfully at:", output_path)


print("\n⏳ Simulating Airflow DAG Steps...")

steps = [
    "Extract patient & clinical data",
    "Run data quality checks",
    "Clean and transform data",
    "Join datasets",
    "Store curated output (like S3)",
]

for s in steps:
    print("✔️", s)

CLEANED PATIENT DATA:


/tmp/ipython-input-2153351087.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_patients["age"].fillna(df_patients["age"].mean(), inplace=True)
/tmp/ipython-input-2153351087.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpl

,patient_id,name,age,gender
0,101,John Doe,35.000000,M
1,102,Alice Smith,42.000000,F
2,103,Robert Lee,29.000000,M
3,104,Maria Gomez,35.333333,F


CLEANED CLINICAL DATA:


,record_id,patient_id,diagnosis,visit_date
0,1,101,Diabetes,2024-01-10
1,2,102,Hypertension,2024-01-12
2,3,103,Unknown,2024-01-15



🔗 Joining datasets...
FINAL CURATED HEALTHCARE DATA:


,patient_id,name,age,gender,record_id,diagnosis,visit_date
0,101,John Doe,35.0,M,1,Diabetes,2024-01-10
1,102,Alice Smith,42.0,F,2,Hypertension,2024-01-12
2,103,Robert Lee,29.0,M,3,Unknown,2024-01-15



📁 Data saved successfully at: /content/curated_healthcare_data.csv

⏳ Simulating Airflow DAG Steps...
✔️ Extract patient & clinical data
✔️ Run data quality checks
✔️ Clean and transform data
✔️ Join datasets
✔️ Store curated output (like S3)
